In [ ]:
%pip install requests
%pip install beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup


# Definisci l'URL della pagina web da scaricare
url = 'https://www.immobiliare.it/vendita-case/roma/?criterio=prezzo&ordine=asc'

# Effettua una richiesta HTTP alla pagina web
response = requests.get(url)
data = {}
# Verifica se la richiesta è andata a buon fine
for i in range(1, 10):

    url += ('&pag=' + str(i))
    response = requests.get(url)
    if response.status_code == 200:
        # Parsing del contenuto HTML utilizzando BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        
        print('Titolo della pagina:', soup.title.text)
        # Trova tutti gli elementi HTML che contengono i titoli delle notizie (questo è un esempio, potrebbe variare a seconda del sito)
        superficie = soup.find_all('li', class_='nd-list__item in-feat__item', attrs={'aria-label': "superficie"})
        prices = soup.find_all('div', class_='in-reListCardPrice')
        # Find all 'a' elements within the specified 'div' element
        links = soup.find_all('div', class_=['nd-mediaObject__content in-reListCard__content', 'nd-mediaObject__content in-reListCard__content is-spaced'])

        # Iterate over each link
        for link, m2, price in zip(links, superficie, prices):
            a_tag = link.find('a')
            if a_tag is not None:
                href = a_tag.get('href')  # Get the 'href' attribute
                title = a_tag.get('title')  # Get the 'title' attribute
                price_in_span = price.find('span')
                if price_in_span is not None:
                    price_text = price_in_span.text
                    int_price = int(price_text.replace("€", "").replace(".", "").replace(" ", "").replace("da", "").split(",")[0])
                # Add the data to the dictionary
                data[title] = {'Link': href,'Prezzo al m2' : (int_price / int(m2.text.replace("m²", "").replace(".", ""))), 'Superficie': m2.text, 'Price': price_text}

        # Print the data

    else:
        print('Errore nella richiesta HTTP:', response.status_code)

Sort Data by Prezzo al m2

In [ ]:
sorted_data = sorted(data.items(), key=lambda x: x[1]['Prezzo al m2'])

for title, info in sorted_data:
    print(f'Title: {title}, Link: {info["Link"]}, Prezzo al m²: {info["Prezzo al m2"]}, Superficie: {info["Superficie"]}, Price: {info["Price"]}')


In [ ]:
html_content = "<html><body><table>"

# Add table headers
html_content += "<tr><th>Title</th><th>Link</th><th>Prezzo al m²</th><th>Superficie</th><th>Price</th></tr>"

for title, info in sorted_data:
    value = info['Prezzo al m2'] if info['Prezzo al m2'] < 1 else round(info['Prezzo al m2'])
    html_content += f'<tr><td>{title}</td><td><a href="{info["Link"]}">Link</a></td><td>{value} €/m² </td><td>{info["Superficie"]}</td><td>{info["Price"]}</td></tr>'

html_content += "</table></body></html>"

# Write the HTML content to a file
with open('output.html', 'w', encoding='utf-8') as f:
    f.write(html_content)